# 04_07: SOLUTION - Evaluating your GraphRAG Pipeline

In [1]:
import os
from dotenv import load_dotenv

from langchain.evaluation.qa.eval_chain import QAEvalChain
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [2]:
load_dotenv(dotenv_path='../.env')

URI = os.getenv("NEO4J_URI")
USER = os.getenv("NEO4J_USER")
PWD = os.getenv("NEO4J_PWD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [12]:
enhanced_graph = Neo4jGraph(url=URI, username=USER, password=PWD, enhanced_schema=True)
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4o", temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [13]:
examples = [
    {"query": "What sports is the International Ski And Snowboard Federation responsible for?",
     "answer": "Alpine Skiing, Freestyle Skiing, Snowboarding, Nordic Combined, Ski Jumping, Cross-Country Skiing"},
    {"query": "In what place was Telemark Skiing invented?", 
     "answer": "Telemark Region"},
    {"query": "Who do athletes get help from?",
     "answer": "Coaches, Peer Mentors, and Sports Psychologists"},
]



In [ ]:
graph_chain = GraphCypherQAChain.from_llm(
    graph=enhanced_graph,
    llm=llm,
    verbose=True,
    allow_dangerous_requests=True,
)

In [5]:
def evaluate_graph_rag(graph_chain, examples):

    # Generate predictions by querying the graph
    predictions = []
    for ex in examples:
        graph_response = graph_chain.invoke({"query": ex["query"]})
        predictions.append({"result": graph_response["result"].strip()})

    # Run evaluation
    eval_chain = QAEvalChain.from_llm(llm)
    results = eval_chain.evaluate(examples, predictions)

    # Print output
    correct = 0
    for i, res in enumerate(results):
        print(f"Query: {examples[i]['query']}")
        print(f"Prediction from graph: {predictions[i]['result']}")
        print(f"Gold answer: {examples[i]['answer']}")
        print(f"Grade: {res['results']}")
        print("---")
        if res["results"] == "CORRECT":
            correct += 1

    accuracy = correct / len(examples)
    print(f"Graph QA Accuracy: {accuracy:.2f}")

In [ ]:
evaluate_graph_rag(graph_chain, examples)

In [22]:
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to
answer questions about skiing.
Convert the user's question based on the schema.
When you are presented with query properties such as id's like "alpine touring",
be sure to convert the first letter to capital case, such as "Alpine Touring"
before you run the Cypher query.

NEVER USE THE MENTIONS RELATIONSHIP FOR ANY ANSWERS!

For example, if I were to ask "Tell me about telemark skiing," you should create
a Cypher query that finds all nodes with the id "Telemark Skiing" and then find
all nodes connected to those nodes and use those to forumulate your answer, like this:

MATCH (a {{id: "Telemark Skiing"}})-[r]-(b)
RETURN a, r, b

You MUST use the question to infer the node label.
"What sports is the International Ski and Snowboard Federation responsible for?"
I would create a query like:

MATCH (a {{id: "International Ski and Snowboard Federation"}})-[r:]-(b:Sport)
RETURN a, r, b

Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

graph_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=enhanced_graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    allow_dangerous_requests=True
)

In [23]:
evaluate_graph_rag(graph_chain, examples)



> Entering new GraphCypherQAChain chain...


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: Sport)} {position: line: 2, column: 69, offset: 75} for query: 'cypher\nMATCH (a {id: "International Ski And Snowboard Federation"})-[r]-(b:Sport)\nRETURN a, r, b\n'


Generated Cypher:
cypher
MATCH (a {id: "International Ski And Snowboard Federation"})-[r]-(b:Sport)
RETURN a, r, b

Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Concept {id: "Telemark Skiing"})-[r]-(b:Place)
RETURN a, r, b

Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Person)-[r:BENEFIT_FROM]->(b:Organization)
RETURN a, r, b

Full Context:
[{'a': {'id': 'Beginners'}, 'r': ({'id': 'Beginners'}, 'BENEFIT_FROM', {'id': 'Surf Schools'}), 'b': {'id': 'Surf Schools'}}, {'a': {'id': 'Intermediates'}, 'r': ({'id': 'Intermediates'}, 'BENEFIT_FROM', {'id': 'Surf Schools'}), 'b': {'id': 'Surf Schools'}}]

> Finished chain.
Query: What sports is the International Ski And Snowboard Federation responsible for?
Prediction from graph: I don't know the answer.
Gold answer: Alpine Skiing, Freestyle Skiing, Snowboarding, Nordic Combined, Ski Jumping, Cross-Country Sk

In [ ]:
CYPHER_GENERATION_TEMPLATE = """

You are an expert Neo4j developer. Your job is to generate a Cypher query that 
will traverse exactly 1–3 hops out from your focus node.

Steps:

1. Identify the main entity in the question and Capitalize Multi-Word IDs
2. Write a variable-length MATCH using the new syntax:
   MATCH (a {{id: "{{entity}}"}})-[r]-{{1,3}}(b)
   RETURN a, r, b;

You MUST use {{1,3}} for setting the number of hops instead of [r*0..3].
Do NOT use WHERE length(r) >= 1 AND length(r) <= 3 to specify it either.

Schema:
{schema}

Question:
{question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

enhanced_graph_chain = GraphCypherQAChain.from_llm(
    graph=enhanced_graph,
    llm=llm,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    allow_dangerous_requests=True,
)

In [ ]:
examples = [
    {"query": "What training method does the International Ski and Snowboard Federation use?",
     "answer": "Grass skiing"},
    {"query": "How does research play a role?",
     "answer": "Research improves competition by looking at Surface Science and Friction Reduction and has implications for Competitive Performance and Recreational Enjoyment"},
    {"query": "What country was the men's overall winner of the 2024–25 Fis Alpine Ski World Cup representing?",
     "answer": "Switzerland"},
]

In [ ]:
evaluate_graph_rag(graph_chain, examples)

In [ ]:
CYPHER_GENERATION_TEMPLATE = """

You are an expert Neo4j developer. Your job is to generate a Cypher query that 
will traverse exactly 1–3 hops out from your focus node.

Steps:

1. Identify the main entity in the question and Capitalize Multi-Word IDs
2. Write a variable-length MATCH using the new syntax:
   MATCH (a {{id: "{{entity}}"}})-[r]-{{1,3}}(b)
   RETURN a, r, b;

You MUST use {{1,3}} for setting the number of hops instead of [r*0..3].
Do NOT use WHERE length(r) >= 1 AND length(r) <= 3 to specify it either.

Schema:
{schema}

Question:
{question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

enhanced_graph_chain = GraphCypherQAChain.from_llm(
    graph=enhanced_graph,
    llm=llm,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    allow_dangerous_requests=True,
)

In [ ]:
evaluate_graph_rag(enhanced_graph_chain, examples)

In [ ]:
ANSWER_SYNTHESIS_TEMPLATE = """
You have a list of variable‑length paths from Neo4j in the form:

(path = [(a)-[:REL1]->(X)-[:REL2]->(b), …])

Your job: extract the intermediate node X and the two relationship names REL1 and REL2,
then produce one sentence of the form:

“The {a.id} {rel1_verb} on {X.id} that {rel2_verb} {b.id}.”

Example:

Paths:
1. (Skiing Community)-[INNOVATE]-(Techniques)-[ENHANCE]-(Safety)

Answer:
The Skiing Community innovates on Techniques that enhance Safety.

Now, given:
{paths}

Answer:
"""

answer_prompt = PromptTemplate(
    template=ANSWER_SYNTHESIS_TEMPLATE,
    input_variables=["paths"],
)

full_graph_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=enhanced_graph,
    cypher_prompt=cypher_generation_prompt,
    answer_prompt=answer_prompt,      # ← override the summarizer
    verbose=True,
    allow_dangerous_requests=True,
)

In [ ]:
evaluate_graph_rag(full_graph_chain, examples)